In [ ]:
# 2025-12-17 against production
import pandas as pd
from csae_pyutils import gsheet_to_df
from apis_core.apis_relations.models import EventWork
from apis_core.apis_vocabularies.models import EventWorkRelation
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm

In [ ]:
df = pd.read_csv("/home/csae8092/repos/burgtherater-event-crawler/data/events.csv")

data = []
for i, ndf in df.groupby("title"):
    item = {"title": i, "events": len(ndf)}
    item["ids"] = ndf["_id"].unique().tolist()
    data.append(item)


df = pd.DataFrame(data)
df = df.sort_values("events", ascending=False)

enriched_df = gsheet_to_df("1EVjKfDhme5y4rz2rVrSURa9YwlVxcSuEjpmlIVKBhZ0")
df = df.merge(enriched_df[["title", "work-pmb-id"]], on="title", how="left")
df = df.dropna(subset=["work-pmb-id"])

In [ ]:
df.head()

In [ ]:
relation_type = EventWorkRelation.objects.get(name="ist Aufführung von")

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    uris = row["ids"]
    try:
        work = Work.objects.get(id=row["work-pmb-id"])
    except:
        print(row["work-pmb-id"])
        continue
    for x in uris:
        uri = f"https://kulturerbe.burgtheater.at/event/{x}"
        try:
            match = Uri.objects.get(uri=uri)
        except ObjectDoesNotExist:
            continue
        event = Event.objects.get(id=match.entity.id)
        EventWork.objects.get_or_create(
            relation_type=relation_type,
            related_event=event,
            related_work=work,
            start_date_written=event.start_date_written,
            end_date_written=event.start_date_written,
        )
        

In [ ]:
for x in uris:
    print(x)